This notebook explores Gröbner bases of knapsack problems with
binary and with integer variables. The idea is to find the points
at which the two differ (Markov basis construction, truncation, etc).

With a good enough description of these differences, it will hopefully
be possible to propose relevant optimizations for the binary case.

In [1]:
using MIPMatrixTools.IPInstances

A = [15 18 19 17 20]
b = [45]
C = [1 1 1 1 1]
u = [nothing for i in 1:5]

integer_instance = IPInstance(A, b, C, u)
println("Integer instance:")
println(integer_instance.model)

u_binary = [1 for i in 1:5]
binary_instance = IPInstance(A, b, C, u_binary)
println("Binary instance:")
println(binary_instance.model)

Integer instance:


Min -x[1] - x[2] - x[3] - x[4] - x[5]
Subject to
 

15 x[1] + 18 x[2] + 19 x[3] + 17 x[4] + 20 x[5] + x[6] = 45.0
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[4] ≥ 0.0
 x[5] ≥ 0.0
 x[6] ≥ 0.0

Binary instance:


Min -x[1] - x[2] - x[3] - x[4] - x[5]
Subject to
 15 x[1] + 18 x[2] + 19 x[3] + 17 x[4] + 20 x[5] + x[6] = 45.0
 x[1] + x[7] = 1.0
 x[2] + x[8] = 1.0
 x[3] + x[9] = 1.0
 x[4] + x[10] = 1.0
 x[5] + x[11] = 1.0
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[4] ≥ 0.0
 x[5] ≥ 0.0
 x[6] ≥ 0.0
 x[7] ≥ 0.0
 x[8] ≥ 0.0
 x[9] ≥ 0.0
 x[10] ≥ 0.0
 x[11] ≥ 0.0



First, I will compare the initial Markov bases of these two knapsacks
(group relaxations).

Result: the bases are essentially the same in both cases.

In [2]:
using IPGBs.Markov

integer_markov = Markov.initialize_project_and_lift(integer_instance)
println("Integer Markov basis:")
println(integer_markov.markov)

binary_markov = Markov.initialize_project_and_lift(binary_instance)
println("Binary Markov basis:")
println(binary_markov.markov)

Integer Markov basis:


[[1, 0, 0, 0, 0, -15], [0, 1, 0, 0, 0, -18], [0, 0, 1, 0, 0, -19], [0, 0, 0, 1, 0, -17], [0, 0, 0, 0, 1, -20]]
Binary Markov basis:
[[1, 0, 0, 0, 0, -15, -1, 0, 0, 0, 0], [0, 1, 0, 0, 0, -18, 0, -1, 0, 0, 0], [0, 0, 1, 0, 0, -19, 0, 0, -1, 0, 0], [0, 0, 0, 1, 0, -17, 0, 0, 0, -1, 0], [0, 0, 0, 0, 1, -20, 0, 0, 0, 0, -1]]


In [9]:
using IPGBs.Markov

full_int_markov = markov_basis(integer_instance)
println("Full Markov basis: ")
println(full_int_markov)

full_bin_markov = markov_basis(binary_instance)
println("Full Markov basis: ")
println(full_bin_markov)

Full Markov basis: 
[[1, 0, 0, 0, 0, -15], [0, 1, 0, 0, 0, -18], [0, 0, 1, 0, 0, -19], [0, 0, 0, 1, 0, -17], [0, 0, 0, 0, 1, -20]]
Full Markov basis: 
[[1, 0, 0, 0, 0, -15, -1, 0, 0, 0, 0], [0, 1, 0, 0, 0, -18, 0, -1, 0, 0, 0], [0, 0, 1, 0, 0, -19, 0, 0, -1, 0, 0], [0, 0, 0, 1, 0, -17, 0, 0, 0, -1, 0], [0, 0, 0, 0, 1, -20, 0, 0, 0, 0, -1]]


┌ Debug: Starting to compute Markov basis for 
│   instance = min [-1.0 -1.0 -1.0 -1.0 -1.0 0.0] 
[15, 18, 19, 17, 20, 1] = 45 

└ @ IPGBs.Markov /home/gmlangeloh/.julia/dev/IPGBs/src/Markov.jl:328
┌ Debug: Computing Markov basis using the Simple Markov algorithm
└ @ IPGBs.Markov /home/gmlangeloh/.julia/dev/IPGBs/src/Markov.jl:287
┌ Debug: Starting to compute Markov basis for 
│   instance = min [-1.0 -1.0 -1.0 -1.0 -1.0 0.0 0.0 0.0 0.0 0.0 0.0] 
[15, 18, 19, 17, 20, 1, 0, 0, 0, 0, 0] = 45 
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0] = 1 
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0] = 1 
[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0] = 1 
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0] = 1 
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1] = 1 
0 <= x1 <= 1
0 <= x2 <= 1
0 <= x3 <= 1
0 <= x4 <= 1
0 <= x5 <= 1

└ @ IPGBs.Markov /home/gmlangeloh/.julia/dev/IPGBs/src/Markov.jl:328
┌ Debug: Computing Markov basis using the Simple Markov algorithm
└ @ IPGBs.Markov /home/gmlangeloh/.julia/dev/IPGBs/src/Markov.jl:287


In [12]:
using IPGBs

integer_gb = groebner_basis(integer_instance)
println("Integer GB of size: ", length(integer_gb))
println("Integer GB: ", integer_gb)

binary_gb = groebner_basis(binary_instance)
println("Binary GB of size: ", length(binary_gb))
println("Binary GB: ", binary_gb)

┌ Debug: Starting to compute Markov basis for 
│   instance = min [-1.0 -1.0 -1.0 -1.0 -1.0 0.0] 
[15, 18, 19, 17, 20, 1] = 45 

└ @ IPGBs.Markov /home/gmlangeloh/.julia/dev/IPGBs/src/Markov.jl:328
┌ Debug: Computing Markov basis using the Simple Markov algorithm
└ @ IPGBs.Markov /home/gmlangeloh/.julia/dev/IPGBs/src/Markov.jl:287
┌ Debug: Starting to compute Gröbner basis for: 
│   markov_basis = [[1, 0, 0, 0, 0, -15], [0, 1, 0, 0, 0, -18], [0, 0, 1, 0, 0, -19], [0, 0, 0, 1, 0, -17], [0, 0, 0, 0, 1, -20]]
└ @ IPGBs /home/gmlangeloh/.julia/dev/IPGBs/src/IPGBs.jl:232
┌ Debug: Truncating generating set with algorithm: Simple
└ @ IPGBs.Buchberger /home/gmlangeloh/.julia/dev/IPGBs/src/Buchberger.jl:112
┌ Debug: Autorreduced binomial
│   original = [0, 0, 0, 0, -1, 20] : c[1]
│   result = [-1, 0, 0, 0, 1, -5] : c[0]
│   reduced_to_zero = false
│   changed = true
└ @ IPGBs.BinomialSets /home/gmlangeloh/.julia/dev/IPGBs/src/BinomialSets.jl:379
┌ Debug: Autorreduced binomial
│   original = [0,

┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 6
│   pair.j = 3
│   element_i = [-1, 1, 0, 0, 0, -3, 1, -1, 0, 0, 0] : c[0]
│   element_j = [0, 0, -1, 0, 0, 19, 0, 0, 1, 0, 0] : c[1]
│   binomial = [1, -1, -1, 0, 0, 22, -1, 1, 1, 0, 0] : c[1]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187
┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 6
│   pair.j = 4
│   element_i = [-1, 1, 0, 0, 0, -3, 1, -1, 0, 0, 0] : c[0]
│   element_j = [0, 0, 0, -1, 0, 17, 0, 0, 0, 1, 0] : c[1]
│   binomial = [1, -1, 0, -1, 0, 20, -1, 1, 0, 1, 0] : c[1]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187
┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 6
│   pair.j = 5
│   element_i = [-1, 1, 0, 0, 0, -3, 1, -1, 0, 0, 0] : c[0]
│   element_j = [0, 0, 0, 0, -1, 20, 0, 0, 0, 0, 1] : c[1]
│   binomial = [1, -1, 0, 0, -1, 23, -1, 1, 0, 0, 1] : c[1]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187
┌ De

┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 10
│   pair.j = 7
│   element_i = [0, -1, 0, 1, 0, 1, 0, 1, 0, -1, 0] : c[0]
│   element_j = [-1, 0, 1, 0, 0, -4, 1, 0, -1, 0, 0] : c[0]
│   binomial = [-1, 1, 1, -1, 0, -5, 1, -1, -1, 1, 0] : c[0]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187
┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 10
│   pair.j = 8
│   element_i = [0, -1, 0, 1, 0, 1, 0, 1, 0, -1, 0] : c[0]
│   element_j = [0, -1, 1, 0, 0, -1, 0, 1, -1, 0, 0] : c[0]
│   binomial = [0, 0, -1, 1, 0, 2, 0, 0, 1, -1, 0] : c[0]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187
┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 10
│   pair.j = 9
│   element_i = [0, -1, 0, 1, 0, 1, 0, 1, 0, -1, 0] : c[0]
│   element_j = [-1, 0, 0, 1, 0, -2, 1, 0, 0, -1, 0] : c[0]
│   binomial = [-1, 1, 0, 0, 0, -3, 1, -1, 0, 0, 0] : c[0]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187
┌ D

┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 13
│   pair.j = 7
│   element_i = [0, -1, 0, 0, 1, -2, 0, 1, 0, 0, -1] : c[0]
│   element_j = [-1, 0, 1, 0, 0, -4, 1, 0, -1, 0, 0] : c[0]
│   binomial = [-1, 1, 1, 0, -1, -2, 1, -1, -1, 0, 1] : c[0]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187
┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 13
│   pair.j = 8
│   element_i = [0, -1, 0, 0, 1, -2, 0, 1, 0, 0, -1] : c[0]
│   element_j = [0, -1, 1, 0, 0, -1, 0, 1, -1, 0, 0] : c[0]
│   binomial = [0, 0, -1, 0, 1, -1, 0, 0, 1, 0, -1] : c[0]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187
┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 13
│   pair.j = 9
│   element_i = [0, -1, 0, 0, 1, -2, 0, 1, 0, 0, -1] : c[0]
│   element_j = [-1, 0, 0, 1, 0, -2, 1, 0, 0, -1, 0] : c[0]
│   binomial = [-1, 1, 0, 1, -1, 0, 1, -1, 0, -1, 1] : c[0]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:18

Integer GB of size: 5
Integer GB: [[-1, 0, 0, 0, 0, 15], [-1, 0, 0, 0, 1, -5], [-1, 0, 0, 1, 0, -2], [-1, 0, 1, 0, 0, -4], [-1, 1, 0, 0, 0, -3]]
Binary GB of size: 15
Binary GB: [[-1, 0, 0, 0, 0, 15, 1, 0, 0, 0, 0], [-1, 0, 0, 0, 1, -5, 1, 0, 0, 0, -1], [-1, 0, 0, 1, 0, -2, 1, 0, 0, -1, 0], [-1, 0, 1, 0, 0, -4, 1, 0, -1, 0, 0], [-1, 1, 0, 0, 0, -3, 1, -1, 0, 0, 0], [0, -1, 0, 0, 0, 18, 0, 1, 0, 0, 0], [0, -1, 0, 0, 1, -2, 0, 1, 0, 0, -1], [0, -1, 0, 1, 0, 1, 0, 1, 0, -1, 0], [0, -1, 1, 0, 0, -1, 0, 1, -1, 0, 0], [0, 0, -1, 0, 0, 19, 0, 0, 1, 0, 0], [0, 0, -1, 0, 1, -1, 0, 0, 1, 0, -1], [0, 0, -1, 1, 0, 2, 0, 0, 1, -1, 0], [0, 0, 0, -1, 0, 17, 0, 0, 0, 1, 0], [0, 0, 0, -1, 1, -3, 0, 0, 0, 1, -1], [0, 0, 0, 0, -1, 20, 0, 0, 0, 0, 1]]


┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 15
│   pair.j = 12
│   element_i = [0, 0, 0, -1, 1, -3, 0, 0, 0, 1, -1] : c[0]
│   element_j = [-1, 0, 0, 0, 1, -5, 1, 0, 0, 0, -1] : c[0]
│   binomial = [-1, 0, 0, 1, 0, -2, 1, 0, 0, -1, 0] : c[0]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187
┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 15
│   pair.j = 13
│   element_i = [0, 0, 0, -1, 1, -3, 0, 0, 0, 1, -1] : c[0]
│   element_j = [0, -1, 0, 0, 1, -2, 0, 1, 0, 0, -1] : c[0]
│   binomial = [0, -1, 0, 1, 0, 1, 0, 1, 0, -1, 0] : c[0]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187
┌ Debug: Eliminated by GCD / cancellation
│   pair.i = 15
│   pair.j = 14
│   element_i = [0, 0, 0, -1, 1, -3, 0, 0, 0, 1, -1] : c[0]
│   element_j = [0, 0, -1, 0, 1, -1, 0, 0, 1, 0, -1] : c[0]
│   binomial = [0, 0, -1, 1, 0, 2, 0, 0, 1, -1, 0] : c[0]
└ @ IPGBs.GBAlgorithms /home/gmlangeloh/.julia/dev/IPGBs/src/GBAlgorithms.jl:187


In [13]:
#while !Markov.is_finished(integer_markov)
#    l = length(integer_markov.sigma)
#    integer_markov = Markov.next(integer_markov, truncation_type = :Heuristic)
#    println("Lifted ", l, " of integer markov, new basis:")
#    println(integer_markov.markov)
#end
#
#while !Markov.is_finished(binary_markov)
#    l = length(binary_markov.sigma)
#    integer_markov = Markov.next(binary_markov, truncation_type = :Heuristic)
#    println("Lifted ", l, " of binary markov, new basis:")
#    println(binary_markov.markov)
#end